In [18]:
import tkinter as tk
from nltk.tokenize import word_tokenize
import pickle

# Load the Baseline and Proposed models
baseline_model_filename = "/Users/eliezer/Documents/School/NLP_Translation/Trained Model/Baseline_fi_hi_translation_model.pkl"
proposed_model_filename = "/Users/eliezer/Documents/School/NLP_Translation/Trained Model/Proposed_fi_hi_translation_model.pkl"

with open(baseline_model_filename, 'rb') as model_file:
    baseline_t = pickle.load(model_file)

with open(proposed_model_filename, 'rb') as model_file:
    proposed_t = pickle.load(model_file)

print("Both models loaded successfully")

# Generate vocabulary from training data
tokenized_stores = {'fi_train': [], 'hi_train': []}

for key in tokenized_stores:
    file_name = "/Users/eliezer/Documents/School/NLP_Translation/TrainS/" + str(key)[3:] + "." + str(key)[0:2]
    with open(file_name) as load:
        sentences = load.read().split('\n')

    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)

# Making the vocabulary
fi_words = {}
hi_words = {}

for key in tokenized_stores:
    if str(key)[0] == 'f':
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in fi_words:
                    fi_words[word] += 1
                else:
                    fi_words[word] = 1
    else:
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in hi_words:
                    hi_words[word] += 1
                else:
                    hi_words[word] = 1

fi_vocab = len(fi_words)
hi_vocab = len(hi_words)

print("Vocabulary sizes - Filipino:", fi_vocab, "Hiligaynon:", hi_vocab)

# Function to translate the input sentence using a given model
def translate(model, direction):
    input_sentence = text_input.get("1.0", tk.END).strip()
    tokenized_input = word_tokenize(input_sentence)
    
    translated_sentence = []
    
    if direction == 'fi_to_hi':
        for fi_word in tokenized_input:
            best_hi_word = None
            max_prob = 0
            
            for hi_word in hi_words:
                if (fi_word, hi_word) in model and model[(fi_word, hi_word)] > max_prob:
                    best_hi_word = hi_word
                    max_prob = model[(fi_word, hi_word)]
            
            if best_hi_word:
                translated_sentence.append(best_hi_word)
            else:
                translated_sentence.append(fi_word)  # Use the original word if no translation is found
    
    elif direction == 'hi_to_fi':
        for hi_word in tokenized_input:
            best_fi_word = None
            max_prob = 0
            
            for fi_word in fi_words:
                if (fi_word, hi_word) in model and model[(fi_word, hi_word)] > max_prob:
                    best_fi_word = fi_word
                    max_prob = model[(fi_word, hi_word)]
            
            if best_fi_word:
                translated_sentence.append(best_fi_word)
            else:
                translated_sentence.append(hi_word)  # Use the original word if no translation is found
    
    return " ".join(translated_sentence)

# Function to update the output labels with translations
def update_translations():
    direction = lang_var.get()
    baseline_translation = translate(baseline_t, direction)
    proposed_translation = translate(proposed_t, direction)
    
    baseline_output.config(state=tk.NORMAL)  # Enable editing to update text
    baseline_output.delete(1.0, tk.END)
    baseline_output.insert(tk.END, baseline_translation)
    baseline_output.config(state=tk.DISABLED)  # Disable editing

    proposed_output.config(state=tk.NORMAL)  # Enable editing to update text
    proposed_output.delete(1.0, tk.END)
    proposed_output.insert(tk.END, proposed_translation)
    proposed_output.config(state=tk.DISABLED)  # Disable editing

# Function to clear the input field and output labels
def clear_fields():
    text_input.delete("1.0", tk.END)
    baseline_output.config(state=tk.NORMAL)  # Enable editing to clear text
    baseline_output.delete(1.0, tk.END)
    baseline_output.config(state=tk.DISABLED)  # Disable editing

    proposed_output.config(state=tk.NORMAL)  # Enable editing to clear text
    proposed_output.delete(1.0, tk.END)
    proposed_output.config(state=tk.DISABLED)  # Disable editing

# Setup Tkinter window
root = tk.Tk()
root.title("Fi-Hi Translator")
root.geometry("1000x400")  # Set window size
root.resizable(False, False)  # Make window non-resizable

# Create frames for layout
input_frame = tk.Frame(root)
input_frame.grid(row=0, column=0, padx=20, pady=20, sticky="n")

center_frame = tk.Frame(root)
center_frame.grid(row=0, column=1, padx=20, pady=20, sticky="n")

output_frame = tk.Frame(root)
output_frame.grid(row=0, column=2, padx=20, pady=20, sticky="n")

# Title for the input box
input_title = tk.Label(input_frame, text="Enter Text to Translate:", font=("Helvetica", 14, "bold"))
input_title.grid(row=0, column=0, pady=(0, 10))

# Input field
text_input = tk.Text(input_frame, width=50, height=23)  # Adjust the height here
text_input.grid(row=1, column=0, pady=10)

# Combine radio buttons and buttons into the center_frame
lang_var = tk.StringVar(value='fi_to_hi')  # Default to Filipino to Hiligaynon

fi_to_hi_radio = tk.Radiobutton(center_frame, text="Filipino to Hiligaynon", variable=lang_var, value='fi_to_hi')
fi_to_hi_radio.grid(row=0, column=0, pady=5)

hi_to_fi_radio = tk.Radiobutton(center_frame, text="Hiligaynon to Filipino", variable=lang_var, value='hi_to_fi')
hi_to_fi_radio.grid(row=1, column=0, pady=5)

translate_button = tk.Button(center_frame, text="Translate", command=update_translations)
translate_button.grid(row=2, column=0, pady=10)

clear_button = tk.Button(center_frame, text="Clear", command=clear_fields)
clear_button.grid(row=3, column=0, pady=10)

# Output labels for Baseline and Proposed models
baseline_label = tk.Label(output_frame, text="Proposed Translation:", font=("Helvetica", 12, "bold"))
baseline_label.grid(row=0, column=0, pady=(10, 0))

baseline_output = tk.Text(output_frame, height=10, width=50, wrap="word", state=tk.DISABLED)
baseline_output.grid(row=1, column=0, pady=(0, 20))

proposed_label = tk.Label(output_frame, text="Baseline Translation:", font=("Helvetica", 12, "bold"))
proposed_label.grid(row=2, column=0, pady=(10, 0))

proposed_output = tk.Text(output_frame, height=10, width=50, wrap="word", state=tk.DISABLED)
proposed_output.grid(row=3, column=0, pady=(0, 20))

root.mainloop()

Both models loaded successfully
Vocabulary sizes - Filipino: 3232 Hiligaynon: 3412
